plan: 
with script, make a string of the following format that includes all haplotypes and also filters on both cwd version 2 and 3 (must be 'C' (common) for both):

https://www.ebi.ac.uk/ipd/imgt/hla/alleles/?query=or(and(eq(cwd.version_3.Total,%22C%22),eq(cwd.version_2,%22C%22),startsWith(name,%22A*24:02%22)),and(eq(cwd.version_3.Total,%22C%22),eq(cwd.version_2,%22C%22),startsWith(name,%22A*01:02%22)))

paste resulting url string into browser and then download the result list mapping IPD Accession (what we need for getting jsons) to allele name.
then loop over all the accessions to get json urls: https://www.ebi.ac.uk/cgi-bin/ipd/api/allele/[HLA00002]
    format: https://www.ebi.ac.uk/cgi-bin/ipd/api/allele/[accession]

In [2]:
import numpy as np
import pandas as pd
import json
from Bio.Seq import Seq
import math
import urllib.request

In [3]:
def readHLA(filename):
    """
    takes in filename of HLA locus xlsx, outputs list of most frequent alleles
    an allele is defined as frequent if it occurs in more than 1% of the population for any ethnic group 
    """
    df = pd.read_excel(filename)
    freq_alleles_list = []
    cutoff = 0.01
    for index, row in df.iterrows():
        # print(row["A"])
        if max(row[1], row[3], row[5], row[7], row[9], row[11], row[13], row[15], row[17], row[19], row[21], row[23], row[25], row[27], row[29], row[31], row[33], row[35], row[37], row[39], row[41], row[43], row[45], row[47], row[49], row[51]) >= cutoff:
            # freq_alleles_list.append(row[0])
            if row[0][-1] == "g":
                freq_alleles_list.append(row[0][:-1])
            else:
                freq_alleles_list.append(row[0])
    return freq_alleles_list

In [ ]:
freq_alleles = {}
freq_alleles["A"] = readHLA("../HLA_allele_freq_sheets/A.xlsx")
freq_alleles["B"] = readHLA("../HLA_allele_freq_sheets/B.xlsx")
freq_alleles["C"] = readHLA("../HLA_allele_freq_sheets/C.xlsx")
freq_alleles["DQB1"] = readHLA("../HLA_allele_freq_sheets/DQB1.xlsx")
freq_alleles["DRB1"] = readHLA("../HLA_allele_freq_sheets/DRB1.xlsx")
# freq_alleles is a dict mapping strings of gene (eg. "A", "B") to list of common haplotype names (strings)

In [ ]:
freq_alleles

In [ ]:
### overfilters 
# prefix = "https://www.ebi.ac.uk/ipd/imgt/hla/alleles/?query=or("
# a_str = str()
# for allele_name in freq_alleles["A"]:
#     #print(allele_name)
#     s = f",and(eq(cwd.version_3.Total,%22C%22),eq(cwd.version_2,%22C%22),startsWith(name,%22{allele_name}%22))"
#     a_str+=s
# a_str = a_str[1:]

# a_str = prefix+a_str+")"
# a_str

'https://www.ebi.ac.uk/ipd/imgt/hla/alleles/?query=or(and(eq(cwd.version_3.Total,%22C%22),eq(cwd.version_2,%22C%22),startsWith(name,%22A*30:04%22)),and(eq(cwd.version_3.Total,%22C%22),eq(cwd.version_2,%22C%22),startsWith(name,%22A*24:03%22)),and(eq(cwd.version_3.Total,%22C%22),eq(cwd.version_2,%22C%22),startsWith(name,%22A*24:02%22)),and(eq(cwd.version_3.Total,%22C%22),eq(cwd.version_2,%22C%22),startsWith(name,%22A*11:01%22)),and(eq(cwd.version_3.Total,%22C%22),eq(cwd.version_2,%22C%22),startsWith(name,%22A*34:02%22)),and(eq(cwd.version_3.Total,%22C%22),eq(cwd.version_2,%22C%22),startsWith(name,%22A*32:01%22)),and(eq(cwd.version_3.Total,%22C%22),eq(cwd.version_2,%22C%22),startsWith(name,%22A*01:01%22)),and(eq(cwd.version_3.Total,%22C%22),eq(cwd.version_2,%22C%22),startsWith(name,%22A*03:02%22)),and(eq(cwd.version_3.Total,%22C%22),eq(cwd.version_2,%22C%22),startsWith(name,%22A*68:01%22)),and(eq(cwd.version_3.Total,%22C%22),eq(cwd.version_2,%22C%22),startsWith(name,%22A*23:01%22)),and(eq

In [ ]:
## only using CWD version 2 to filter seems to work well 
prefix = "https://www.ebi.ac.uk/ipd/imgt/hla/alleles/?query=or("
a_str = str()
for allele_name in freq_alleles["A"]:
    #print(allele_name)
    s = f",and(eq(cwd.version_2,%22C%22),startsWith(name,%22{allele_name}%22))"
    a_str+=s
a_str = a_str[1:]

a_str = prefix+a_str+")"
a_str

'https://www.ebi.ac.uk/ipd/imgt/hla/alleles/?query=or(and(eq(cwd.version_2,%22C%22),startsWith(name,%22A*30:04%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22A*24:03%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22A*24:02%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22A*11:01%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22A*34:02%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22A*32:01%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22A*01:01%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22A*03:02%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22A*68:01%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22A*23:01%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22A*11:02%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22A*74:01%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22A*02:05%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22A*26:01%22)),and(eq(cwd.version_2,%22C%22),startsWith(name,%22A*30:01%22)),and(eq(cwd.versi

In [ ]:
n_to_a = pd.read_csv("HLA_A_name_to_accession.txt", sep="\t")
n_to_a 
name_to_accession = dict(zip(n_to_a['name'], n_to_a['accession']))
name_to_accession

{'A*30:04:01:01': 'HLA00092',
 'A*24:03:01:01': 'HLA00053',
 'A*24:02:01:01': 'HLA00050',
 'A*11:01:01:01': 'HLA00043',
 'A*34:02:01:01': 'HLA00109',
 'A*32:01:01:01': 'HLA00101',
 'A*01:01:01:01': 'HLA00001',
 'A*03:02:01:01': 'HLA00040',
 'A*68:01:01:01': 'HLA00115',
 'A*68:01:02:01': 'HLA00116',
 'A*23:01:01:01': 'HLA00048',
 'A*11:02:01:01': 'HLA00044',
 'A*74:01:01:01': 'HLA00127',
 'A*02:05:01:01': 'HLA00010',
 'A*26:01:01:01': 'HLA00073',
 'A*30:01:01:01': 'HLA00089',
 'A*34:01:01:01': 'HLA00108',
 'A*26:03:01:01': 'HLA00075',
 'A*66:01:01:01': 'HLA00112',
 'A*03:01:01:01': 'HLA00037',
 'A*02:11:01:01': 'HLA00016',
 'A*02:02:01:01': 'HLA00007',
 'A*02:03:01:01': 'HLA00008',
 'A*33:01:01:01': 'HLA00104',
 'A*30:02:01:01': 'HLA00090',
 'A*24:07:01:01': 'HLA00057',
 'A*29:02:01:01': 'HLA00086',
 'A*25:01:01:01': 'HLA00071',
 'A*68:02:01:01': 'HLA00117',
 'A*33:03:01:01': 'HLA00106',
 'A*36:01:01:01': 'HLA00110',
 'A*29:01:01:01': 'HLA00085',
 'A*02:01:01:01': 'HLA00005',
 'A*02:01:

In [1]:
import urllib.request, json 
with urllib.request.urlopen("https://www.ebi.ac.uk/cgi-bin/ipd/api/allele/HLA00002") as url:
    data = json.load(url)
    print(data)
# https://www.ebi.ac.uk/cgi-bin/ipd/api/allele/HLA00002

{'accession': 'HLA00002', 'allele_history': [{'name': 'A*0102', 'release_version': '1.05.0', 'sversion': 1}, {'name': 'A*0102', 'release_version': '1.06.0', 'sversion': 1}, {'name': 'A*0102', 'release_version': '1.07.0', 'sversion': 1}, {'name': 'A*0102', 'release_version': '1.08.0', 'sversion': 1}, {'name': 'A*0102', 'release_version': '1.09.0', 'sversion': 1}, {'name': 'A*0102', 'release_version': '1.10.0', 'sversion': 1}, {'name': 'A*0102', 'release_version': '1.11.0', 'sversion': 1}, {'name': 'A*0102', 'release_version': '1.12.0', 'sversion': 1}, {'name': 'A*0102', 'release_version': '1.13.0', 'sversion': 1}, {'name': 'A*0102', 'release_version': '1.14.0', 'sversion': 1}, {'name': 'A*0102', 'release_version': '1.15.0', 'sversion': 1}, {'name': 'A*0102', 'release_version': '1.16.0', 'sversion': 1}, {'name': 'A*0102', 'release_version': '2.00.0', 'sversion': 1}, {'name': 'A*0102', 'release_version': '2.01.0', 'sversion': 1}, {'name': 'A*0102', 'release_version': '2.02.0', 'sversion':